Week 2 Notebook: Data Formats and Exploration
===============================================================

Particle physics uses a data format (and analysis ecosystem) based on the `ROOT` library: https://root.cern.ch/.
`ROOT` is a framework for data processing, born at CERN, at the heart of the research on high-energy physics. 
A `ROOT` file is a compressed binary file in which we can save objects of any type.
There are python bindings built into `ROOT`, so called `PyROOT`, but for now we won't discuss this.

Recently, a different library called `uproot` has been developed allowing python users to do `ROOT` I/O directly: https://uproot.readthedocs.io/en/latest/.
`uproot` is a reader and a writer of the `ROOT` file format using only `Python` and `Numpy`. 
Unlike the standard C++ `ROOT` implementation, uproot is only an I/O library, primarily intended to stream data into machine learning libraries in `Python`. 
Unlike `PyROOT` and `root_numpy`, uproot does not depend on C++ `ROOT`. 
Instead, it uses `Numpy` to cast blocks of data from the `ROOT` file as `Numpy` arrays.
It can also make jagged or awkward arrays based on this library: https://github.com/scikit-hep/awkward-1.0 {cite:p}`Pivarski:2020qcb`.


For a tutorial on `uproot`, see: https://hsf-training.github.io/hsf-training-uproot-webpage/ or here: https://github.com/jpivarski-talks/2020-07-13-pyhep2020-tutorial.

In [ ]:
import uproot

## Remote files
One nice feature of `ROOT` and `uproot` is the ability to read a file remotely. 
This is done using the `XRootD` library: https://xrootd.slac.stanford.edu/.
`XRootD` is a generic suite for fast, low latency and scalable data access, which can serve natively any kind of data, organized as a hierarchical filesystem-like namespace, based on the concept of directory. 

## XRootD filenames
We specify that this is a remote file accessed via `XRootD` using the `root://` protocol in front of the filename. 
The redirector `eospublic.cern.ch` specifies that we're accessing data from the CERN open data repository. 
And finally the rest of the file name `/eos/opendata/cms/datascience/HiggsToBBNtupleProducerTool/HiggsToBBNTuple_HiggsToBB_QCD_RunII_13TeV_MC/train/ntuple_merged_10.root` specifies the exact file.

In [ ]:
f = uproot.open('root://eospublic.cern.ch//eos/opendata/cms/datascience/HiggsToBBNtupleProducerTool/HiggsToBBNTuple_HiggsToBB_QCD_RunII_13TeV_MC/train/ntuple_merged_10.root')

## Trees
One common object is a tree. 
Trees in `ROOT` are basically just tables of information. 
Trees are composed of branches, which are the columns of the table. 
The rows usually represent events (individual bunch crossings).
However, in this case, each row represents a jet (a localized collection of particles within a single event).

First we assign the tree to a variable (named `tree` here).
We can see how many rows (jets) are contained in the tree, by checking its `num_entries`.

In [ ]:
tree = f['deepntuplizer/tree']
print(tree.num_entries)

Let's list the contents (branches) of the tree.

In [ ]:
tree.show()

## A note on "jaggedness"

Some branches are listed has having an interpretation of `asdtype('>f4')` while some others are listed as `asjagged(asdtype('>f4'), 10)`.
The former means there is only one number per jet.
The latter means there may a variable number per jet. 

First, let's get just look at non-jagged arrays, starting with the ground truth labels.

In [ ]:
# Returns a dictionary
labels = tree.arrays(['label_QCD_b',
                      'label_QCD_bb', 
                      'label_QCD_c', 
                      'label_QCD_cc', 
                      'label_QCD_others', 
                      'label_H_bb',
                      'sample_isQCD'], 
                     entry_stop=20000,
                     library='np')
labels

In [ ]:
# label QCD: require the sample to be QCD and any of the QCD flavors
label_QCD = labels['sample_isQCD'] * (labels['label_QCD_b'] + \
                                      labels['label_QCD_bb'] + \
                                      labels['label_QCD_c'] + \
                                      labels['label_QCD_cc'] + \
                                      labels['label_QCD_others'])

# label Hbb
label_Hbb = labels['label_H_bb']

## What is the signal and background?

Signal (Higgs) events for this task look like this: 

$$gg \to X\to HH \to b\bar{b}b\bar{b}$$

<img width = 400px src="http://cms-results.web.cern.ch/cms-results/public-results/publications/B2G-17-006/CMS-B2G-17-006_Figure_001-a.png"/>

Background (QCD) events for this task look like this: 

$$gg \to q \bar{q} / gg / b\bar{b} / gb\bar{b} / ggg / \cdots$$

<img width = 400px src="https://www.physik.uzh.ch/~che/FeynDiag/diagrams/1_pp_scattering/10000013.png"/>

We will look at differentiating signal and background at the "jet" level.

In [ ]:
# number of overlap jets
sum(label_QCD*label_Hbb) 

In [ ]:
# fraction of jets with some truth label defined 
sum(label_QCD+label_Hbb)/len(label_QCD+label_Hbb)

In [ ]:
# jet features, namely transverse momemntum (pt) and soft-drop mass  (sdmass or msd)
jet_features = tree.arrays(['fj_pt', 
                            'fj_sdmass'],
                          entry_stop=20000,
                          library='np')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure()

plt.hist(jet_features['fj_pt'],weights=label_QCD,bins=np.linspace(0,4000,101),density=True,alpha=0.7,label='QCD')
plt.hist(jet_features['fj_pt'],weights=label_Hbb,bins=np.linspace(0,4000,101),density=True,alpha=0.7,label='H(bb)')
plt.xlabel(r'Jet $p_{T}$ [GeV]')
plt.ylabel('Fraction of jets')
plt.legend()

plt.figure()

plt.hist(jet_features['fj_sdmass'],weights=label_QCD,bins=np.linspace(0,300,101),density=True,alpha=0.7,label='QCD')
plt.hist(jet_features['fj_sdmass'],weights=label_Hbb,bins=np.linspace(0,300,101),density=True,alpha=0.7,label='H(bb)')
plt.xlabel(r'Jet $m_{SD}$ [GeV]')
plt.ylabel('Fraction of jets')
plt.legend()

plt.show()

## Jagged arrys

Now let's look at an jagged (or awkward) array, like those related to track features, where there can be a variable number of tracks per jet.

In [ ]:
import awkward as ak
track_features = tree.arrays(['track_pt',
                              'label_H_bb'],
                             entry_stop=20000,
                             library='ak')
track_features

Note the difference between a "flat" array, where these a fixed number per jet (like 1 per jet in the case of jet properties) and a jagged array, where there are a variable number. 

We can demonstrate this by looking at the first jet in the dataset. As we'll see there are 21 tracks, each with their own pt.

In [ ]:
jet_features['fj_pt'][0]

In [ ]:
track_features['track_pt'][0]

Note, behind the scenes, jagged arrays are just like normal numpy arrays, except there's additional structure, retreivable from the `ak.num` function, which tells us the (variable) number of tracks per jet.

In [ ]:
ak.num(track_features['track_pt'])

## Operations with jaggedness

We can plot the full distibution of track pts. But what if we want to find the highest track pt per jet and plot only that?

In [ ]:
plt.figure()
plt.hist(ak.flatten(track_features['track_pt']),bins=np.linspace(0,100,101),density=True,alpha=0.7,label='All tracks')
plt.hist(ak.max(track_features['track_pt'], axis=-1),bins=np.linspace(0,100,101),density=True,alpha=0.7,label=r'Max. $p_{T}$ track per jet')

plt.xlabel(r'Track $p_{T}$')
plt.ylabel('Fraction of tracks')
plt.legend()
plt.show()

## Conversion to regular array

Sometimes we want to turn a jagged array into a regular array to make it easier to accomodate into a machine learning algorithm (like a fully conneted or convolutional neural network). 
The simplest way to do this is to use zero-padding and truncation to "cap" the number of objects at some fixed value, and zero-pad if there are less objects.

To do this, we can first plot how many tracks there are per jet and choose a reasonable number to cap.

In [ ]:
plt.figure()
plt.hist(ak.num(track_features['track_pt']),bins=np.linspace(0,100,101),density=True,alpha=0.7)

plt.xlabel(r'Number of tracks')
plt.ylabel('Fraction of jets')
plt.show()

In this case, 60 seems to be a reasonable number.

In [ ]:
max_tracks = 60
pad_value = 0
a = ak.fill_none(ak.pad_none(track_features['track_pt'], max_tracks, clip=True, axis=-1), pad_value, axis=-1).to_numpy()
print(a.shape)
a

To recreate the previous plot, we can do the following (note: we expect minor differences):

In [ ]:
plt.figure()
plt.hist(a[a>0],bins=np.linspace(0,100,101),density=True,alpha=0.7, label='All tracks')
plt.hist(np.max(a,axis=-1),bins=np.linspace(0,100,101),density=True,alpha=0.7, label=r'Max. $p_{T}$ track per jet')

plt.xlabel(r'Number of tracks')
plt.ylabel('Fraction of jets')
plt.legend()
plt.show()
print(a.nonzero())